In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from category_encoders import TargetEncoder
from cstm_pkg_grp_9.data.sets import pop_target
from joblib import dump
import os

# Load Data

In [2]:
train_data = pd.read_csv("../../data/interim/train_data_cleaned.csv")
test_data = pd.read_csv("../../data/interim/test_data_cleaned.csv")

In [3]:
holiday_data = pd.read_csv("../../data/processed/calendar_events.csv")

# Verify Data

## Train data

In [4]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34720691 entries, 0 to 34720690
Data columns (total 9 columns):
 #   Column         Dtype  
---  ------         -----  
 0   item_id        object 
 1   dept_id        object 
 2   cat_id         object 
 3   store_id       object 
 4   state_id       object 
 5   sales_revenue  float64
 6   year           int64  
 7   month          int64  
 8   day            int64  
dtypes: float64(1), int64(3), object(5)
memory usage: 2.3+ GB


In [5]:
train_data.head()

,item_id,dept_id,cat_id,store_id,state_id,sales_revenue,year,month,day
0,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,5.52,2011,1,29
1,HOBBIES_1_009,HOBBIES_1,HOBBIES,CA_1,CA,3.12,2011,1,29
2,HOBBIES_1_010,HOBBIES_1,HOBBIES,CA_1,CA,0.00,2011,1,29
3,HOBBIES_1_012,HOBBIES_1,HOBBIES,CA_1,CA,0.00,2011,1,29
4,HOBBIES_1_015,HOBBIES_1,HOBBIES,CA_1,CA,2.80,2011,1,29


In [6]:
train_data.shape

(34720691, 9)

## Test data

In [7]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12160986 entries, 0 to 12160985
Data columns (total 9 columns):
 #   Column         Dtype  
---  ------         -----  
 0   item_id        object 
 1   dept_id        object 
 2   cat_id         object 
 3   store_id       object 
 4   state_id       object 
 5   sales_revenue  float64
 6   year           int64  
 7   month          int64  
 8   day            int64  
dtypes: float64(1), int64(3), object(5)
memory usage: 835.0+ MB


In [8]:
test_data.head()

,item_id,dept_id,cat_id,store_id,state_id,sales_revenue,year,month,day
0,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0.00,2015,4,19
1,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0.00,2015,4,19
2,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0.00,2015,4,19
3,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,18.56,2015,4,19
4,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,8.64,2015,4,19


In [9]:
holiday_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167 entries, 0 to 166
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   date        167 non-null    object
 1   event_name  167 non-null    object
 2   event_type  167 non-null    object
dtypes: object(3)
memory usage: 4.0+ KB


In [10]:
holiday_data.head()

,date,event_name,event_type
0,2011-02-06,SuperBowl,Sporting
1,2011-02-14,ValentinesDay,Cultural
2,2011-02-21,PresidentsDay,National
3,2011-03-09,LentStart,Religious
4,2011-03-16,LentWeek2,Religious


# Data Transformation

Creating a column date by combing the year, month and day columns and converting the combined value to a date time object.

In [13]:
train_data['date'] = pd.to_datetime(train_data[['year', 'month', 'day']])
test_data['date'] = pd.to_datetime(test_data[['year', 'month', 'day']])

The objective is to forecast 7 days of total sales revenue across all the stores and items. Thus, the data will be aggregated and the sum of sales revenue for each store and its items per day will be calculated.

In [14]:
train_df_agg = train_data.groupby('date')['sales_revenue'].sum().reset_index()
test_df_agg = test_data.groupby('date')['sales_revenue'].sum().reset_index()

Verifying the count of rows

In [14]:
train_df_agg.count()

date             1541
sales_revenue    1541
dtype: int64

In [15]:
test_df_agg.count()

date             400
sales_revenue    400
dtype: int64

Renaming columns so that they can be used by the forecasting model.

In [16]:
train_df_agg.rename(columns={'date': 'ds', 'sales_revenue': 'y'}, inplace=True)
test_df_agg.rename(columns={'date': 'ds', 'sales_revenue': 'y'}, inplace=True)

Transforming holiday data that can be ingested as parameter in the forecast model.

In [30]:
holiday_data['date'] = pd.to_datetime(holiday_data['date'])
holiday_data.rename(columns={'date': 'ds', 'event_name':'holiday'}, inplace=True)

# Saving Files

In [31]:
folder_path = "../../data/processed"
train_df_agg.to_csv(os.path.join(folder_path, 'train_forecasting.csv'), index=False)
test_df_agg.to_csv(os.path.join(folder_path, 'test_forecasting.csv'), index=False)
holiday_data.to_csv(os.path.join(folder_path, 'holiday_data.csv'), index=False)